# Collecting data from ThermoML xml files

In [ ]:
import os
from ThermoML import ThermoMLBuilder
import polars as pl
import numpy as np
from importlib import reload

## Activity coeficient

Collecting data from files with activity coeficient data

In [ ]:
reload(ThermoMLBuilder)

In [ ]:
dir = os.getcwd()
files = []
with open('ThermoML/binaryAC.txt', 'r') as f:
    files = [os.path.join(dir, 'ThermoML', file.strip()+'.xml')
             for file in f.readlines()]
files[0]

In [ ]:
[data, compounds] = ThermoMLBuilder.build_dataset(files)

In [ ]:
compounds.describe()

In [ ]:
data.describe()

In [ ]:
data.write_parquet('data/datasetAC.parquet')

In [ ]:
compounds.write_parquet('data/compoundsAC.parquet')